# Pandas Methods for Tidying Data

Most real-world data arrives _untidy_. Before you can analyze it, you need to reshape it into a structure that pandas can work with effectively. This lecture maps common untidy patterns to the specific pandas methods that fix them.

Recall the tidy data principles:

- Every row is one observation
- Every column is one variable
- Every cell is one value

When data violates these principles, pandas operations like `groupby()`, filtering, and plotting become awkward or impossible. The solution is reshaping.

In [ ]:
import pandas as pd
import numpy as np

## The Tidying Toolkit

Your primary reshaping tools:

- `melt()` — unpivot columns into rows (wide → long)
- `pivot()` — reshape rows into columns (long → wide)
- `pivot_table()` — pivot with aggregation for duplicates
- `.str.split(expand=True)` — separate combined values into columns

We will work through each of the five common untidy patterns and see which tool solves each one.

## Pattern 1: Column Headers Are Values

This is the most common untidy pattern. Years, months, dates, or categories appear as column names instead of as values in a column.

```text
| Country | 2020 | 2021 | 2022 |
|---------|------|------|------|
| USA     | 100  | 110  | 120  |
| Canada  | 80   | 85   | 90   |
```

The years are _values_ masquerading as column names. You cannot easily filter by year or plot a time series because pandas has no Year column to work with.

In [ ]:
# Create the untidy data
untidy = pd.DataFrame({
    'Country': ['USA', 'Canada'],
    '2020': [100, 80],
    '2021': [110, 85],
    '2022': [120, 90]
})

untidy

### The Solution: `melt()`

The `melt()` function _unpivots_ columns into rows. Think of it as melting those wide columns down into long rows.

Key parameters:

- `id_vars` — columns to keep as-is (the identifiers)
- `var_name` — name for the column that will hold the former headers
- `value_name` — name for the column that will hold the cell values

In the example, we want to convert the year column names into a new column (`Year`) with those values and the original values into a new column (`GDP`). Te rest will remain unchanged.

In [ ]:
tidy = untidy.melt(
    id_vars='Country',
    var_name='Year',
    value_name='GDP'
)

tidy

Now each observation (one country in one year) is its own row. You can filter, group, and plot naturally.

In [ ]:
# Now filtering by year is trivial
tidy[tidy['Year'] == '2021']

In [ ]:
# Convert Year to integer for proper sorting/plotting
tidy['Year'] = tidy['Year'].astype(int)
tidy.sort_values(['Country', 'Year'])

## Pattern 5: Repeated Measures as Columns

This pattern is structurally identical to Pattern 1. Trial numbers, time points, or repeated measurements appear as column headers.

```text
| Subject | Trial1 | Trial2 | Trial3 |
|---------|--------|--------|--------|
| A       | 12.3   | 11.8   | 12.1   |
| B       | 15.2   | 14.9   | 15.0   |
```

The trial numbers are values that belong in a Trial column.

In [ ]:
untidy = pd.DataFrame({
    'Subject': ['A', 'B'],
    'Trial1': [12.3, 15.2],
    'Trial2': [11.8, 14.9],
    'Trial3': [12.1, 15.0]
})

untidy

### Same Solution: `melt()`

In [ ]:
tidy = untidy.melt(
    id_vars='Subject',
    var_name='Trial',
    value_name='Score'
)

tidy

You may want to clean up the Trial column to extract just the number.

In [ ]:
# Extract trial number
tidy['Trial'] = tidy['Trial'].str.replace('Trial', '').astype(int)
tidy

## Pattern 2: Multiple Variables in One Column

Two or more pieces of information are crammed into a single column, typically with a delimiter.

```text
| ID | Gender_Age |
|----|------------|
| 1  | M_25       |
| 2  | F_30       |
| 3  | M_42       |
```

Gender and Age are separate variables that need their own columns.

In [ ]:
untidy = pd.DataFrame({
    'ID': [1, 2, 3],
    'Gender_Age': ['M_25', 'F_30', 'M_42']
})

untidy

### The Solution: `.str.split(expand=True)`

The `.str` accessor provides string methods for Series. The `split()` method with `expand=True` returns a DataFrame where each split piece becomes its own column. Like `str.split` in base Python, but with new tricks!

In [ ]:
# Split returns a DataFrame with expand=True
untidy['Gender_Age'].str.split('_', expand=True)

In [ ]:
# Assign back to new columns
untidy[['Gender', 'Age']] = untidy['Gender_Age'].str.split('_', expand=True)

# Convert Age to numeric
untidy['Age'] = untidy['Age'].astype(int)

# Drop the original combined column
tidy = untidy.drop(columns='Gender_Age')

tidy

## Pattern 3: Variables in Both Rows and Columns

This pattern combines the previous issues. Variable names are encoded in column headers, often with multiple pieces of information.

```text
| Date       | Temp_Morning | Temp_Evening | Humidity_Morning | Humidity_Evening |
|------------|--------------|--------------|------------------|------------------|
| 2024-01-01 | 45           | 52           | 65               | 70               |
| 2024-01-02 | 43           | 50           | 68               | 72               |
```

The column headers encode two things: the variable (Temp or Humidity) and the time of day (Morning or Evening).

In [ ]:
untidy = pd.DataFrame({
    'Date': ['2024-01-01', '2024-01-02'],
    'Temp_Morning': [45, 43],
    'Temp_Evening': [52, 50],
    'Humidity_Morning': [65, 68],
    'Humidity_Evening': [70, 72]
})

untidy

### The Solution: Multi-Step Reshaping

Complex patterns often require multiple operations. Here we need to:

1. Melt to get all values in rows
2. Split the combined column names
3. Pivot to get variables (Temp, Humidity) back as columns

In [ ]:
# Step 1: Melt everything
melted = untidy.melt(
    id_vars='Date',
    var_name='Var_Time',
    value_name='Value'
)

melted

In [ ]:
# Step 2: Split the combined column
melted[['Variable', 'Time']] = melted['Var_Time'].str.split('_', expand=True)

melted

`pivot` is essentially the inverse of `melt`. Here we want the values in the `Variable` column to become column headers, with the corresponding `Value` entries filling those columns.

In [ ]:
# Step 3: Pivot so each variable becomes a column
tidy = melted.pivot_table(
    index=['Date', 'Time'],
    columns='Variable',
    values='Value'
).reset_index()

# Clean up the column names
tidy.columns.name = None

tidy

Now each row is one observation (one date at one time of day), and each variable (Temp, Humidity) has its own column.

## Pattern 4: Multiple Observation Types in One Column

Values and their labels are stored together, often from data entry systems.

```text
| ID | Measurement  |
|----|--------------|
| 1  | Height: 5.8  |
| 1  | Weight: 180  |
| 1  | Age: 30      |
| 2  | Height: 6.1  |
| 2  | Weight: 195  |
| 2  | Age: 28      |
```

The Measurement column contains both the variable name and the value.

In [ ]:
untidy = pd.DataFrame({
    'ID': [1, 1, 1, 2, 2, 2],
    'Measurement': ['Height: 5.8', 'Weight: 180', 'Age: 30',
                    'Height: 6.1', 'Weight: 195', 'Age: 28']
})

untidy

### The Solution: Split then Pivot

In [ ]:
# Split the measurement column
untidy[['Variable', 'Value']] = untidy['Measurement'].str.split(': ', expand=True)

# Convert Value to numeric
untidy['Value'] = pd.to_numeric(untidy['Value'])

untidy

In [ ]:
# Pivot so each measurement type becomes a column
tidy = untidy.pivot(
    index='ID',
    columns='Variable',
    values='Value'
).reset_index()

# Clean up column names
tidy.columns.name = None

tidy

## The Inverse: `pivot()` for Long to Wide

Sometimes data arrives in an overly long format, or you need to restructure for a specific analysis. The `pivot()` function reshapes from long to wide, the opposite of `melt()`.

In [ ]:
# Long format data
long_data = pd.DataFrame({
    'Store': ['A', 'A', 'B', 'B'],
    'Quarter': ['Q1', 'Q2', 'Q1', 'Q2'],
    'Sales': [100, 120, 80, 95]
})

long_data

In [ ]:
# Pivot to wide format
wide_data = long_data.pivot(
    index='Store',
    columns='Quarter',
    values='Sales'
)

wide_data

### `pivot()` vs `pivot_table()`

Use `pivot()` when your index/column combination is unique (no duplicates). If duplicates exist, you need `pivot_table()` which can aggregate them.

In [ ]:
# Data with duplicates
data_with_dupes = pd.DataFrame({
    'Store': ['A', 'A', 'A', 'B'],
    'Quarter': ['Q1', 'Q1', 'Q2', 'Q1'],  # Store A has two Q1 entries
    'Sales': [100, 50, 120, 80]
})

data_with_dupes

In [ ]:
# pivot() would fail here - use pivot_table() with an aggregation function
wide = data_with_dupes.pivot_table(
    index='Store',
    columns='Quarter',
    values='Sales',
    aggfunc='sum'  # or 'mean', 'count', etc.
)

wide

## Decision Framework

When you encounter untidy data, ask these questions:

| If you see...                                               | Use...                                               |
| ----------------------------------------------------------- | ---------------------------------------------------- |
| Column headers that are really values (years, trials, etc.) | `melt()`                                             |
| Multiple variables crammed into one column                  | `.str.split(expand=True)`                            |
| Data that is too long and needs columns                     | `pivot()` or `pivot_table()`                         |
| Complex encoding in column names                            | Multiple steps: `melt()` → `str.split()` → `pivot()` |

The pattern recognition comes with practice. When in doubt, ask yourself: what should each row represent? What should each column represent? Then reshape to match that structure.

## Summary

The core tidying operations:

- `melt(id_vars, var_name, value_name)` — unpivot columns to rows
- `pivot(index, columns, values)` — reshape rows to columns
- `pivot_table(index, columns, values, aggfunc)` — pivot with aggregation
- `.str.split(delimiter, expand=True)` — separate combined values

Most tidying involves `melt()` because human-readable formats tend to be wide. Once your data is tidy, all of pandas' analysis tools work smoothly.